In [1]:
import random
import syft as sy
from syft.core import utils
from syft.core.frameworks.torch import utils as torch_utils
from syft.core.frameworks import encode
from syft.core.frameworks.torch.tensor import _MPCTensor
from syft.core.frameworks.torch.tensor import _GeneralizedPointerTensor
from syft.mpc import spdz
import torch
import torch.nn.functional as F
from torch.autograd import Variable as Var
import json

hook = sy.TorchHook(verbose=True)

me = hook.local_worker
me.is_client_worker = False

bob = sy.VirtualWorker(id="bob", hook=hook, is_client_worker=False)
alice = sy.VirtualWorker(id="alice", hook=hook, is_client_worker=False)
james = sy.VirtualWorker(id="james", hook=hook, is_client_worker=False)

bob.add_workers([alice, james])
alice.add_workers([bob, james])
james.add_workers([bob, alice])

In [2]:
x = torch.LongTensor([[-1,2],[3,4]])
x = x.share(bob, alice)

y = torch.LongTensor([[2,2],[2,2]]).send(bob, alice)

z = x * y

In [3]:
z.get()


-2  4
 6  8
[syft.core.frameworks.torch.tensor.LongTensor of size 2x2]

In [6]:
z.get()


-2  4
 6  8
[syft.core.frameworks.torch.tensor.LongTensor of size 2x2]